<img style="float: left; margin: 30px 15px 15px 15px;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFzQj91sOlkeDFkg5HDbjtR4QJYmLXkfMNig&usqp=CAU" width="400" height="500" /> 
    
    
### <font color='navy'> Modelos no lineales para pronósticos. 

**Nombres:** Xander Gallegos, Sergio Daniel Dueñas.

**Fecha:** 12 de marzo de 2024.

**Expediente** : 737989, XXXXXX.
    
**Profesor:** Oscar David Jaramillo Zuluaga.
    
**Link Github**: <https://github.com/xagallegos/ProyectoConjunto_T5_XGallegos_SDDuenas>

# Tarea 5: Clase 13

## Enunciado de tarea 
> Implementar todas las técnicas vistas en esta clase al siguiente dataset para predecir el precio de las viviendas:
<https://www.kaggle.com/datasets/yasserh/housing-prices-dataset>

Código de solución estudiante 1

In [ ]:
# Código de solución estudiante 1
.
.
.
.


Código de solución estudiante 2

In [ ]:
# Código de solución estudiante 2
.
.
.
.
